In [ ]:
import mdsine2 as md2
import pandas as pd
import numpy as np

# set negative binomial model inference parameters
params = md2.config.NegBinConfig(
  #seed=0, burnin=100, n_samples=200,
  seed=0, burnin=10, n_samples=20,
  checkpoint=10,
  basepath=str("./negbin")
)

dataset = md2.dataset.parse(
  name="david-diet", 
  taxonomy="taxonomy.tsv",  
  reads="reads.tsv", 
  qpcr="qpcr.tsv", 
  metadata="metadata.tsv",
  perturbations="perturbations.tsv"
)

In [ ]:
mcmc_negbin = md2.negbin.build_graph(
    params=params,
    graph_name=dataset.name,
    subjset=dataset
)

mcmc_negbin.run()

###### mcmc_negbin.run()

In [ ]:
from mdsine2.names import STRNAMES

print('a0', md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A0]))
print('a1', md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A1]))

In [ ]:
import pathlib

a0 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A0])['mean']
a1 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A1])['mean']

basepath = pathlib.Path("./mdsine2/david")
basepath.mkdir(exist_ok=True, parents=True)

# Initialize parameters of the model (Seed = 0) burnin=50, total steps=100
params = md2.config.MDSINE2ModelConfig(
    basepath=str(basepath), 
    seed=0,
#    burnin=50, 
    burnin=10,
#    n_samples=100, 
    n_samples=20,
    negbin_a0=a0, negbin_a1=a1, 
    checkpoint=10
)

mcmc = md2.initialize_graph(params=params, graph_name=dataset.name, subjset=dataset)
mcmc = md2.run_graph(mcmc, crash_if_error=True)

https://github.com/gerberlab/MDSINE2#computation-and-visualization-of-forward-simulation-of-learned-glv-model-for-each-otu

In [ ]:
n_taxa = 310
n_time = 20
perturbations = [np.ones(n_taxa), np.zeros(n_taxa)]
interactions = np.eye(n_taxa)
starts = [8, 15]
ends = [12, 19]
growths = .01 * np.ones(n_taxa)

dyn = md2.model.gLVDynamicsSingleClustering(growth=growths, interactions=interactions, perturbations=perturbations, perturbation_starts=starts, perturbation_ends=ends)
x = md2.integrate(dynamics=dyn, dt=.01, n_days=1, initial_conditions=np.ones((n_taxa, 1)))
x

In [ ]:
import pickle

#with open('mcmc-david.pickle', 'wb') as handle:
#    pickle.dump(mcmc, handle)

#with open("mcmc-david.pickle", "rb") as f:
#    mcmc2 = pickle.load(f)